<a href="https://colab.research.google.com/github/henrykohl/Machine-Learning-demo-repo/blob/master/case-study/grad01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# [葉子張量和非葉子張量](https://www.modb.pro/db/542980)
連結已失效

## case 1-1

In [ ]:
import torch
# 此時w1是葉子張量
w1 = torch.ones(10, requires_grad=True)

with torch.no_grad():
  w1 *= 0.5


w2 = torch.ones(10, requires_grad=True)

wto = torch.exp(w1)*w2

loss = wto.sum()
loss.backward()

```python
print(w1.is_leaf)
print(w2.is_leaf)
# w1 與 w2 都是葉節點

print(w1.grad)
print(w2.grad)
```
True

True

tensor([1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487,
        1.6487])

tensor([1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487,
        1.6487])

## case 1-2



In [ ]:
# 可改成 (此時w1不是葉子張量)
w1 = torch.ones(10, requires_grad=True)*0.5

w2 = torch.ones(10, requires_grad=True)

# wto = torch.exp(w1)*w2

# loss = wto.sum()
# loss.backward()

In [ ]:
print(w1.is_leaf) # w1 是非葉節點
print(w2.is_leaf) # w2 是葉節點

print(w1.grad) # 非葉子張量，求導結果不會保留，所以會有警告
print(w2.grad)

False
True
None
None


<ipython-input-8-487a018ae770>:4: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(w1.grad)




```python
print(w1.is_leaf) # w1 是非葉節點
print(w2.is_leaf) # w2 是葉節點

print(w1.grad)
print(w2.grad)
```
False

True

None

tensor([1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487, 1.6487,
        1.6487])

...


# [詳解Pytorch中的requires_grad、葉子節點與非葉子節點](https://blog.csdn.net/qq_36429555/article/details/118657440)

## case 2-1

In [ ]:
a=torch.tensor([1.0], requires_grad=True) # 是葉子

b=a+1 # 不是葉子

print(a.is_leaf)
print(a.requires_grad) # 明確設定是True
print(b.is_leaf)
print(b.requires_grad) #


b.retain_grad() # b不是葉子，反向求導時若要使用.grad去顯示，必需要使用retain_grad()
b.backward()
print(b.grad)

True
True
False
True tensor([1.])


In [ ]:
a=torch.tensor([1.0]) # 是葉子

b=a+1 # 依然是葉子，因為不需要對a求導，所以反向求導時，a節點變得沒有意義

print(a.is_leaf)
print(a.requires_grad) # 是False(沒設定是True)
print(b.is_leaf)    # 注意，是True
print(b.requires_grad) #

# 注意，b是葉子，所以b不能調用backward()，此例子很特殊，因為沒有非葉子節點

True
False
True
False


## case 2-2
https://blog.csdn.net/qq_27825451/article/details/95498211

http://nysdy.com/post/pytorch_detach_data/

In [ ]:
a=torch.tensor([1.0], requires_grad=True) # 是葉子

b=a.sigmoid()
# b=a+1 # 不同的運算式

c = b.detach()

"""與detach有關"""
# c.requires_grad = True
# c.sum().backward() # 如果沒加上一行，單單這一行會出現錯誤，用這個分離出來的tensor去求導，會影響backward()，所以出現錯誤

"""與detach無關"""
b.sum().backward()


print(a.grad)

tensor([0.1966])


In [ ]:
"""錯誤示範"""
a=torch.tensor([1.0], requires_grad=True) # 是葉子
print(a.grad)
b=a.sigmoid() # 注意這裡是用.sigmoid(), 如果b=a+1，後面的c進行in place操作，並不會出錯(Why?)
print(b)

c = b.detach()
print(c)
c.zero_() # 使用in-place操作，對c修改，這會會影響backward()，所以出現錯誤

print(c)
print(b)

b.sum().backward()

print(a.grad)

None
tensor([0.7311], grad_fn=<SigmoidBackward0>)
tensor([0.7311])
tensor([0.])
tensor([0.], grad_fn=<SigmoidBackward0>)


RuntimeError: ignored

In [ ]:
""".detach改用.data，backward不會出錯，因為.data後的修改不會被autograd追蹤，但卻得到錯誤的backward"""
a=torch.tensor([1.0], requires_grad=True) # 是葉子
print(a.grad)
b=a.sigmoid()
print(b)

c = b.data # 改變c，autograd不會追蹤到，所以不會報錯
print(c)
c.zero_()

print(c)
print(b)

b.sum().backward()

print(a.grad) # 結果被影影到



None
tensor([0.7311], grad_fn=<SigmoidBackward0>)
tensor([0.7311])
tensor([0.])
tensor([0.], grad_fn=<SigmoidBackward0>)
tensor([0.])


In [ ]:
"""改變sum到c.zero_()之前，backward結果就會是正確的"""
a=torch.tensor([1.0], requires_grad=True) # 是葉子
print(a.grad)
b=a.sigmoid().sum() # sum移到此
print(b)

c = b.data
print(c)
c.zero_()

print(c)
print(b)

b.backward()

print(a.grad)

None
tensor(0.7311, grad_fn=<SumBackward0>)
tensor(0.7311)
tensor(0.)
tensor(0., grad_fn=<SumBackward0>)
tensor([0.1966])


# [PyTorch的Tensor和自动求导](https://lulaoshi.info/machine-learning/neural-network/pytorch-tensor-autograd)


In [ ]:
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

print(x)
print(y)
print(z)
print(out)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)
tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)
tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>)
tensor(27., grad_fn=<MeanBackward0>)


In [ ]:
out.backward() # 反向求導

In [ ]:
print(x.grad)  # 葉節點，有求導值
print(y.grad)  # 非葉節點，求導值不保留，所以是None
print(z.grad)  # 非葉節點，求導值不保留，所以是None
print(out.grad) # 非葉節點，求導值不保留，所以是None

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])
None
None
None


<ipython-input-12-68aa1dcaf5e5>:2: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more informations. (Triggered internally at aten/src/ATen/core/TensorBody.h:489.)
  print(y.grad)
<ipython-input-12-68aa1dcaf5e5>:3: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is being accessed. Its .grad attribute won't be populated during autograd.backward(). If you indeed want the .grad field to be populated for a non-leaf Tensor, use .retain_grad() on the non-leaf Tensor. If you access the non-leaf Tensor by mistake, make sure you access the leaf Tensor instead. See github.com/pytorch/pytorch/pull/30531 for more inf


如果在非葉節點使用retain_grad()
```python
x = torch.ones(2, 2, requires_grad=True)
y = x + 2
z = y * y * 3
out = z.mean()

y.retain_grad()
z.retain_grad()
out.retain_grad()

out.backward()

print(x.grad)  
print(y.grad)   
print(z.grad)  
print(out.grad)
```
tensor([[4.5000, 4.5000], <br>
     [4.5000, 4.5000]])

tensor([[4.5000, 4.5000], <br>
        [4.5000, 4.5000]])

tensor([[0.2500, 0.2500], <br>
        [0.2500, 0.2500]])
        
tensor(1.)
